In [1]:
import numpy as np

def transform_point_4x3(point, matrix):
    return np.array([
        matrix[0] * point[0] + matrix[4] * point[1] + matrix[8] * point[2] + matrix[12],
        matrix[1] * point[0] + matrix[5] * point[1] + matrix[9] * point[2] + matrix[13],
        matrix[2] * point[0] + matrix[6] * point[1] + matrix[10] * point[2] + matrix[14]
    ])

def compute_cov_2d(mean, focal_x, focal_y, tan_fovx, tan_fovy, cov3d, viewmatrix):
    t = transform_point_4x3(mean, viewmatrix)

    limx = 1.3 * tan_fovx
    limy = 1.3 * tan_fovy
    txtz = t[0] / t[2]
    tytz = t[1] / t[2]
    t[0] = np.clip(txtz, -limx, limx) * t[2]
    t[1] = np.clip(tytz, -limy, limy) * t[2]

    J = np.array([
        [focal_x / t[2], 0.0, -(focal_x * t[0]) / (t[2] ** 2)],
        [0.0, focal_y / t[2], -(focal_y * t[1]) / (t[2] ** 2)],
        [0.0, 0.0, 0.0]
    ])

    W = np.array([
        [viewmatrix[0], viewmatrix[4], viewmatrix[8]],
        [viewmatrix[1], viewmatrix[5], viewmatrix[9]],
        [viewmatrix[2], viewmatrix[6], viewmatrix[10]]
    ])

    T = np.dot(W, J)

    Vrk = np.array([
        [cov3d[0], cov3d[1], cov3d[2]],
        [cov3d[1], cov3d[3], cov3d[4]],
        [cov3d[2], cov3d[4], cov3d[5]]
    ])

    cov = np.dot(np.dot(T.T, Vrk), T)

    cov[0, 0] += 0.3
    cov[1, 1] += 0.3

    return np.array([cov[0, 0], cov[0, 1], cov[1, 1]])

# 테스트 데이터
def main():
    mean = np.array([1.0, 2.0, 3.0])
    focal_x = 100.0
    focal_y = 100.0
    tan_fovx = 0.5
    tan_fovy = 0.5
    cov3d = np.array([0.1, 0.01, 0.01, 0.2, 0.02, 0.3])
    viewmatrix = np.eye(4).flatten()

    result = compute_cov_2d(mean, focal_x, focal_y, tan_fovx, tan_fovy, cov3d, viewmatrix)
    print("Result:", result)

if __name__ == "__main__":
    main()


Result: [111.41111111  11.11111111 222.52222222]
